# Recitation 10

UTDallas CS 4375, taught by Dr. Ruozzi

Recitation with Jim Amato

#### Jim notes to self for imaginary eventual update:

* Better math. That was one of the two big points of confusion:
  * What does the math in the slides mean?
  * How does it translate to the stuff in this notebook?
* Make "Why so weird looking" a footnote with a link
* Store test/train data in a class?
* Store eigenstuff in a class?
* Have PCA Classifier class, input is just k (# eigenvectors to use)
* Have visualization tools that operate on that class to show eigenvectors, centroids, prototypes
* Revisit part 4; organize math there better so it meshes better with the code
* Rearrange starting around (7)
  * Classification results with k=2 (as currently exists)
  * Visualizations here?
  * Then go to k=3,
  * More visualizations; compare 2 to 3?
  * Then all k from 2-20ish
  * Plot of train acc and test acc
  * Then sidebar 1: Wiggle Room (top n results)
  * Last sidebar 2: Why so weird looking footnote (comparing this toy to full MNIST)
* Add 3D Visualization tool
* Widget that allows changing k and shows eigenvectors/centroids/prototypes ?
* Add Q's & A's throughout?
* Other extensions to this? Using PCA with X method?
* Merge with W9? Or better balance placement of math stuff (between the two)?
  * Flesh out math stuff in here?
* Provide templates to be filled out while going over this (instead of completed code)
  * SubtractMeanScaler
  * EigenStuff
  * Centroids
  * PCA Classifier (dot products vs multiplication for projection)

#$\color{blue}{\text{Today:}}$

* Any requests?
* PCA continued: PCA on Fun Data

#$\color{blue}{\text{Problem 1:}} \text{ PCA on MNIST}$

##$\color{blue}{\text{1.}} \text{ Quick PCA Review}$

\<Refer to last week's Colab notebook\>

##$\color{blue}{\text{2.}} \text{ MNIST}$

The MNIST dataset is literally everywhere. You've certainly seen it. It's a default ("toy") dataset in scikit-learn. More information is available:
* In [Scikit-Learn's Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html)
* In a [Scikit-Learn Tutorial](https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py)
* At the [UCI Archive](https://archive.ics.uci.edu/dataset/80/optical+recognition+of+handwritten+digits)

First we load the MNIST Dataset and split into test and training sets.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
print(digits.data.shape)

X = digits.data
y = digits.target

print(f"Number of samples: {len(X)}")
print(f"Number of features per sample: {X.shape[1]}")
print(f"Unique classes in the dataset: {np.unique(y)}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Let's look at some examples.

In [ ]:
# Display a grid of sample digits
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))
for ax, image, label in zip(axes.ravel(), digits.images, digits.target):
    ax.imshow(image, cmap=plt.cm.gray_r)
    ax.set_title(f'Label: {label}')
    ax.axis('off')
plt.tight_layout()
plt.show()

##$\color{blue}{\text{3.}} \text{ Normalization}$

Normalizing data, especially in the context of PCA, is essential. For PCA, we often ensure our data has zero mean.



This is in the slides on slide 11. We construct the matrix $W\in\mathbb{R}^{n\times p}$ whose $i^\text{th}$ column is:
$$ x_i - \frac{\sum_j x_j}{p}$$

I'm going to deviate a bit and have my data in rows (like usual) instead.

In [ ]:
class MeanSubtractionScaler:
    def __init__(self, X):
        self.mean = np.mean(X, axis=0)

    def scale(self, X):
        return X - self.mean

    def unscale(self, s_X):
        return s_X + self.mean

In [ ]:
scaler = MeanSubtractionScaler(X_train)
X_train_scaled = scaler.scale(X_train)
X_test_scaled = scaler.scale(X_test)

 Let's look at what happens when we subtract the mean from our images.

In [ ]:
fig, axarr = plt.subplots(1, 1, figsize=(15, 3))

for i in range(1):
    # Original images
    axarr.imshow(scaler.mean.reshape(8, 8), cmap='gray_r')
    axarr.axis('off')
    axarr.set_title(f'Mean Digit')

plt.tight_layout()
plt.show()

In [ ]:
fig, axarr = plt.subplots(2, 5, figsize=(15, 6))

# Display 5 original and mean-subtracted images
for i in range(5):
    # Original images
    axarr[0, i].imshow(X_train[i].reshape(8, 8), cmap='gray_r')
    axarr[0, i].axis('off')
    axarr[0, i].set_title(f'Original Image {i+1}')

    # Mean-subtracted images
    axarr[1, i].imshow(X_train_scaled[i].reshape(8, 8), cmap='gray_r', vmin=-16, vmax=16)
    axarr[1, i].axis('off')
    axarr[1, i].set_title(f'Mean-Subtracted Image {i+1}')

plt.tight_layout()
plt.show()

They only sorta look like numbers now. However, this ensures that our principle components are truly the directions of maximum ***variance***.

##$\color{blue}{\text{4.}} \text{ Eigenvectors}$

We're going to use the SVD method, meaning we don't compute the covariance matrix and use the data directly.

In [ ]:
def compute_eigen_via_svd(X):
    # Compute the SVD
    U, s, Vt = np.linalg.svd(X, full_matrices=False)

    # The eigenvalues are the squared singular values
    eigenvalues = s**2

    # The eigenvectors are the columns of Vt
    eigenvectors = Vt

    # Sort the eigenvalues and eigenvectors by descending eigenvalue
    sort_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sort_indices]
    eigenvectors = eigenvectors[:, sort_indices]

    return eigenvalues, eigenvectors

eigenvalues, eigenvectors = compute_eigen_via_svd(X_train_scaled)

This is exactly what we did last week.

We're now going to get $Q_k$, the top $k$ eigenvectors, as described on slide 9 this way:

The Frobenius norm is a matrix norm given by
$$ ||A||_F=\sqrt{\sum_{i=1}^n \sum_{j=1}^n|A_{ij}|^2}$$

$Q_kD_kQ_K^\text{T}$ is the best rank $k$ approximation of the symetric matrix $A$ with respect to the Frobenius norm
$$Q_kD_kQ_k^\text{T} = \underset{B\in\mathbb{R}^{n\times n}\text{s.t.}\text{rank}(B)=k}{\mathrm{argmin}}||A-B||_F$$

That's scary looking math, but it just means that if our data $\mathbf{X}$ has $d$ dimensions ($\mathbf{x}\in\mathbb{R}^d$) then $Q_k$ is going to be a matrix of size $d \times k$:
$$Q_k = \begin{bmatrix}\mathbf{v}_1 & \mathbf{v}_2 & \dots & \mathbf{v}_k\end{bmatrix}$$

In [ ]:
def get_Qk(k_):
    return eigenvectors[:k_]

Q_2 = get_Qk(2)

##$\color{blue}{\text{5.}} \text{ Projecting Data}$

We now have a new space into which we can project our data and get a more simple representation of it. We do this simply with the dot product.

In [ ]:
X_train_projected = np.dot(X_train_scaled, Q_2.T)
X_test_projected = np.dot(X_test_scaled, Q_2.T)

What did the data look like before? What does it look like in the new space?

In [ ]:
print("One datum in the old space:")
print(X_train[0])
print("\nOne datum in the new space:")
print(X_train_projected[0])

We can much more easily imagine data points in this new space. Let's plot several.

In [ ]:
import seaborn as sns
colors = sns.color_palette('hsv', 10)

plt.figure(figsize=(12, 8))

for digit, color in enumerate(colors):
    # Extract data points of the current class
    indices = y_train == digit
    plt.scatter(X_train_projected[indices, 0],
                X_train_projected[indices, 1],
                color=color,
                s=50, alpha=0.6, label=str(digit))

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Projection (2D) of Digits Data')
plt.legend(title='Digits', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


##$\color{blue}{\text{6.}} \text{ Centroids}$

Great. We can imagine a point in the middle of any of those blobs of color. That's the ***centroid***. We're going to determine the centroid for each of these classes.

In [ ]:
centroids = {}
for i in range(10):  # since there are 10 classes (0 through 9)
    centroids[i] = np.mean(X_train_projected[y_train == i], axis=0)

for digit, centroid in centroids.items():
    print(f"Centroid for digit {digit}: {centroid}")

In [ ]:
plt.figure(figsize=(12, 8))

# Plotting the PCA-Projected Data
for digit, color in enumerate(colors):
    # Extract data points of the current class
    indices = y_train == digit
    plt.scatter(X_train_projected[indices, 0],
                X_train_projected[indices, 1],
                color=color,
                s=50, alpha=0.6, label=str(digit))

# Plotting the Centroids
for digit, centroid in centroids.items():
    plt.scatter(centroid[0],
                centroid[1],
                color=colors[digit],
                marker='s', edgecolor='black',
                linewidth=1, s=200, label=f"Centroid {digit}")

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Projection (2D) of Digits Data with Centroids')
plt.legend(title='Digits', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


##$\color{blue}{\text{7.}} \text{ Classification}$

Now we can try to classify data based on its location. We're going to simply take the projected data and find the centroid closest to it. There are certainly more advanced methods than this (you'll use one on the next homework), but it's a great intuitive start.

In [ ]:
def centroid_class(X, centroids):
    centroid_np = np.array(list(centroids.values()))
    diff = X[:, np.newaxis] - centroid_np
    data_distances = np.linalg.norm(diff, axis=2)
    predicted_classes = np.argsort(data_distances, axis=1)
    return predicted_classes[:, :1]

def is_label_the_top_prediction(y_true, y_pred):
    matches = (y_true[:, np.newaxis] == y_pred)
    return np.any(matches, axis=1)

def accuracy(y_true, y_pred):
    matches = (y_true[:, np.newaxis] == y_pred)
    correct = np.sum(np.any(matches, axis=1))
    total = y_true.shape[0]
    return correct / total

In [ ]:
y_train_pred = centroid_class(X_train_projected, centroids)
y_test_pred = centroid_class(X_test_projected, centroids)

print(f"On the training set, we get: {accuracy(y_train, y_train_pred)*100:.01f}% correct")
print(f"On the testing set, we get: {accuracy(y_test, y_test_pred)*100:.01f}% correct")

That's not bad at all! We'd get 10% with random guessing.

##$\color{blue}{\text{7.1}} \text{ Experiment: More Eigenvectors}$

We did that with only the top two eigenvectors. What happens if we use more? Let's try 3.

In [ ]:
Q_3 = get_Qk(3)

X_train_projected_3 = np.dot(X_train_scaled, Q_3.T)
X_test_projected_3 = np.dot(X_test_scaled, Q_3.T)

centroids_3 = {}
for i in range(10):  # since there are 10 classes (0 through 9)
    centroids_3[i] = np.mean(X_train_projected_3[y_train == i], axis=0)

for digit, centroid in centroids_3.items():
    print(f"Centroid for digit {digit}: {centroid}")

In [ ]:
y_train_pred = centroid_class(X_train_projected_3, centroids_3)
y_test_pred = centroid_class(X_test_projected_3, centroids_3)

print(f"On the training set, we get: {accuracy(y_train, y_train_pred)*100:.01f}% correct")
print(f"On the testing set, we get: {accuracy(y_test, y_test_pred)*100:.01f}% correct")

Way better!

How many eigenvectors should we use? We can look at our eigenvalues to see where they drop off.

In [ ]:
plt.figure(figsize=(10, 6))
plt.semilogy(eigenvalues, '-o')
plt.title('Eigenvalues of the Covariance Matrix')
plt.xlabel('Eigenvalue Index')
plt.ylabel('Eigenvalue (log scale)')
plt.ylim(bottom=1)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

It's on a log scale, where we see a pretty smooth fall off after the first 4. So I think 4 is probably good. Let's check performance.

In [ ]:
from tabulate import tabulate

def many_q_k():
    res = []
    for i in range(2, 20):
        Q_k = get_Qk(i)

        X_train_projected_k = np.dot(X_train_scaled, Q_k.T)
        X_test_projected_k = np.dot(X_test_scaled, Q_k.T)

        centroids_k = {}
        for j in range(10):  # since there are 10 classes (0 through 9)
            centroids_k[j] = np.mean(X_train_projected_k[y_train == j], axis=0)

        y_train_pred = centroid_class(X_train_projected_k, centroids_k)
        y_test_pred = centroid_class(X_test_projected_k, centroids_k)

        res.append(dict(k=i,
                        train_acc=accuracy(y_train, y_train_pred),
                        test_acc=accuracy(y_test, y_test_pred)))

    print(tabulate(res, headers="keys"))
many_q_k()

##$\color{blue}{\text{7.2}} \text{ Experiment: More Wiggle Room}$

WIP

In [ ]:
# def centroid_class(X, centroids, first_n):
#     centroid_np = np.array(list(centroids.values()))
#     diff = X[:, np.newaxis] - centroid_np
#     data_distances = np.linalg.norm(diff, axis=2)
#     predicted_classes = np.argsort(data_distances, axis=1)
#     return predicted_classes[:, :first_n]

# def is_label_in_top_predictions(y_true, y_pred):
#     matches = (y_true[:, np.newaxis] == y_pred)
#     return np.any(matches, axis=1)

# def accuracy(y_true, y_pred):
#     matches = (y_true[:, np.newaxis] == y_pred)
#     correct = np.sum(np.any(matches, axis=1))
#     total = y_true.shape[0]
#     return correct / total

##$\color{blue}{\text{8.1}} \text{ Visualization: Eigenvectors}$

In [ ]:
n_eigenvectors = 3
Q_u = None
X_train_projected_u = None
X_test_projected_u = None
centroids_u = None

def update_eigenstuff(num_eigenvectors = 10):
    global Q_u, X_train_projected_u, X_test_projected_u, centroids_u
    Q_u = get_Qk(num_eigenvectors)

    X_train_projected_u= np.dot(X_train_scaled, Q_u.T)
    X_test_projected_u= np.dot(X_test_scaled, Q_u.T)

    centroids_u= {}
    for i in range(10):  # since there are 10 classes (0 through 9)
        centroids_u[i] = np.mean(X_train_projected_u[y_train == i], axis=0)

In [ ]:
update_eigenstuff(n_eigenvectors)

In [ ]:
# Create a figure and subplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))

for i, ax in enumerate(axes.ravel()):
    if i < n_eigenvectors:
        # temp = Q_u[i] + scaler.mean
        eigenimage = Q_u[i].reshape(8, 8)
        ax.imshow(eigenimage, cmap='gray_r')
        ax.set_title(f'Eigenvector {i+1}')
    ax.axis('off')  # Turn off axis numbers and ticks

plt.tight_layout()
plt.show()

##$\color{blue}{\text{8.2}} \text{ Visualization: Centroids}$

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))

for digit, ax in zip(centroids, axes.ravel()):
    # Project centroid back to the original space
    projected_centroid = np.dot(centroids_u[digit], Q_u)
    projected_centroid = scaler.unscale(projected_centroid)
    centroid_image = projected_centroid.reshape(8, 8)
    ax.imshow(centroid_image, cmap='gray_r')
    ax.set_title(f'Digit {digit}')
    ax.axis('off')  # Turn off axis numbers and ticks

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))

for digit, ax in zip(centroids, axes.ravel()):
    # Project centroid back to the original space
    projected_centroid = np.dot(centroids_u[digit], Q_u)
    projected_centroid = scaler.unscale(projected_centroid)
    centroid_image = projected_centroid.reshape(8, 8)
    ax.imshow(centroid_image, cmap='gray_r')
    ax.set_title(f'Digit {digit}')
    ax.axis('off')  # Turn off axis numbers and ticks

plt.tight_layout()
plt.show()

##$\color{blue}{\text{8.3}} \text{ Visualization: Prototypes}$

##$\color{blue}{\text{9}} \text{ Wrap-Up and Extra}$

tbd

###$\color{blue}{\text{2.}} \text{ Sidebar: Why so weird looking?}$

They look... recognizable, but also a bit weird. Why is that? Here's the description available at [UCI](https://archive.ics.uci.edu/dataset/80/optical+recognition+of+handwritten+digits):

"We used preprocessing programs made available by NIST to extract normalized bitmaps of handwritten digits from a preprinted form. From a total of $43$ people, $30$ contributed to the training set and different $13$ to the test set. $32\times32$ bitmaps are divided into nonoverlapping blocks of $4\times4$ and the number of on pixels are counted in each block. This generates an input matrix of $8\times8$ where each element is an integer in the range $[0\dots16]$. This reduces dimensionality and gives invariance to small distortions."

(This isn't the full MNIST, which is 32x32 pixel images, this is the toy dataset from Scikit-Learn.)

In [ ]:
import requests

# Image URL
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/f7/MnistExamplesModified.png"

# Fetch the image using requests
response = requests.get(image_url)

# Make sure the request was successful
response.raise_for_status()

local_image_filename = "local_mnist.png"

# Save the image locally
with open(local_image_filename, "wb") as f:
    f.write(response.content)

print("Image downloaded and saved as local_image.png")

In [ ]:
from PIL import Image

# Replace this path with the path to your image
image = Image.open(local_image_filename)

# Define the left, upper, right, and lower pixel coordinates for cropping
# This will extract the top-left 28x28 pixels
cropped_image = image.crop((0, 0, 28, 28))

# Convert to grayscale
image_gray = cropped_image.convert('L')
threshold = 128
image_binary = image_gray.point(lambda p: 255 if p > threshold else 0)

# Process the image to get the 7x7 matrix
matrix_7x7 = np.zeros((7, 7))

for i in range(0, 28, 4):
    for j in range(0, 28, 4):
        # Extract the 4x4 block from the image
        block = image_binary.crop((j, i, j+4, i+4))

        # Count the number of 'on' pixels. We'll consider a pixel 'on' if its value is below 128 (midway in 0-255 scale)
        on_pixel_count = np.sum(np.array(block) < 128)

        # Store the count in the 8x8 matrix
        matrix_7x7[i//4, j//4] = on_pixel_count

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 4))
ax1.imshow(cropped_image)
ax1.set_title('Original Image')
ax1.axis('off')

ax2.imshow(image_binary, cmap='gray')
ax2.set_title('Black and White Image')
ax2.axis('off')

ax3.imshow(matrix_7x7, cmap='gray_r', vmin=0, vmax=16)
ax3.set_title('Processed 7x7 Representation')
ax3.axis('off')
plt.tight_layout()
plt.show()

###$\text{Template}$

###$\text{Completed}$

###$\text{Showcase Code}$

###$\text{Showcase}$

# Resources

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
print(digits.data.shape)

X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# display_digit_idx = 2
# display_digit = X[0].reshape(8, 8)
# plt.imshow(display_digit, cmap="gray_r")
# plt.show()

class OurOwnScaler:
    def __init__(self, X):
        self.mean = np.mean(X, axis=0)

    def scale(self, X):
        return X - self.mean

    def unscale(self, s_X):
        return s_X + self.mean

scaler = OurOwnScaler(X_train)
X_tr_sc = scaler.scale(X_train)
X_te_sc = scaler.scale(X_test)

U, s, VT = np.linalg.svd(X_tr_sc, full_matrices=False)

k = 3

top_k_vectors = VT[:k]

X_train_projected = np.dot(X_tr_sc, top_k_vectors.T)

centroids = {}
for i in range(10):  # since there are 10 classes (0 through 9)
    centroids[i] = np.mean(X_train_projected[y_train == i], axis=0)

for digit, centroid in centroids.items():
    print(f"Centroid for digit {digit}: {centroid}")

X_test_projected = np.dot(X_te_sc, top_k_vectors.T)

def prototype_class(X, centroids, first_n):
    centroid_np = np.array(list(centroids.values()))
    diff = X[:, np.newaxis] - centroid_np
    data_distances = np.linalg.norm(diff, axis=2)
    predicted_classes = np.argsort(data_distances, axis=1)
    return predicted_classes[:, :first_n]

h_first_n = 2
y_train_pred = prototype_class(X_train_projected, centroids, h_first_n)
y_test_pred = prototype_class(X_test_projected, centroids, h_first_n)

def is_label_in_top_predictions(y_true, y_pred):
    matches = (y_true[:, np.newaxis] == y_pred)
    return np.any(matches, axis=1)

def accuracy(y_true, y_pred):
    matches = (y_true[:, np.newaxis] == y_pred)
    correct = np.sum(np.any(matches, axis=1))
    total = y_true.shape[0]
    return correct / total

# in_top_predictions = is_label_in_top_predictions(y_test, y_test_pred)
# in_top_predictions = is_label_in_top_predictions(y_train, y_train_pred)

# Count how many are correctly classified within the top 'first_n' predictions
print(accuracy(y_train, y_train_pred))
print(accuracy(y_test, y_test_pred))

# # Number of eigenvectors to display
# num_eigenvectors = 10

# # Create a figure and subplots
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))

# for i, ax in enumerate(axes.ravel()):
#     if i < num_eigenvectors:
#         eigenimage = top_k_vectors[i].reshape(8, 8)
#         ax.imshow(eigenimage, cmap='gray')
#         ax.set_title(f'Eigenvector {i+1}')
#     ax.axis('off')  # Turn off axis numbers and ticks

# plt.tight_layout()
# plt.show()


# # Number of centroids (should be 10 for the digits dataset)
# num_centroids = len(centroids)

# # Create a figure and subplots
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 4))

# for digit, ax in zip(centroids, axes.ravel()):
#     # Project centroid back to the original space
#     projected_centroid = np.dot(centroids[digit], top_k_vectors)
#     projected_centroid = scaler.unscale(projected_centroid)
#     centroid_image = projected_centroid.reshape(8, 8)
#     ax.imshow(centroid_image, cmap='gray_r')
#     ax.set_title(f'Digit {digit}')
#     ax.axis('off')  # Turn off axis numbers and ticks

# plt.tight_layout()
# plt.show()

